# Retrieval part
## Design VectorDB using ChromaDB
## On Spider Train Data

## Installation

In [ ]:
!apt install libomp-dev
!pip install faiss
!pip install faiss-gpu
!pip install -U sentence-transformers
!pip install openai
!pip install chromadb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 0s (5,546 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 123586 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a

# Reading dataset

In [ ]:
import json
import re
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from openai import OpenAI
import chromadb

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Load the Spider dataset
train_path = "/content/drive/MyDrive/spider/spider/train_spider.json"
with open(train_path, 'r') as f:
    spider_train_data = json.load(f)

In [ ]:
spider_train_data[0]

{'db_id': 'department_management',
 'query': 'SELECT count(*) FROM head WHERE age  >  56',
 'query_toks': ['SELECT',
  'count',
  '(',
  '*',
  ')',
  'FROM',
  'head',
  'WHERE',
  'age',
  '>',
  '56'],
 'query_toks_no_value': ['select',
  'count',
  '(',
  '*',
  ')',
  'from',
  'head',
  'where',
  'age',
  '>',
  'value'],
 'question': 'How many heads of the departments are older than 56 ?',
 'question_toks': ['How',
  'many',
  'heads',
  'of',
  'the',
  'departments',
  'are',
  'older',
  'than',
  '56',
  '?'],
 'sql': {'from': {'table_units': [['table_unit', 1]], 'conds': []},
  'select': [False, [[3, [0, [0, 0, False], None]]]],
  'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]],
  'groupBy': [],
  'having': [],
  'orderBy': [],
  'limit': None,
  'intersect': None,
  'union': None,
  'except': None}}

## Get OpenAI Embedding

In [ ]:
client_open_ai = OpenAI(api_key='OPEN_AI_API_KEY')

# Function to get embeddings using OpenAI
def get_openai_embedding(text):
    response = client_open_ai.embeddings.create(
                model="text-embedding-ada-002",
                input=text,
            )
    return np.array(response.data[0].embedding)

# Full Implementation and Testing

## Obtain Train Data Embeddings

In [ ]:
questions = [item['question'] for item in spider_train_data]

In [ ]:
# Get embeddings for the original questions
original_question_embeddings = [get_openai_embedding(q) for q in questions]

In [ ]:
len(original_question_embeddings)

7000

In [ ]:
# Save embeddings for later
save_path = "/content/drive/MyDrive/spider/"
np.save(save_path + 'original_question_embeddings.npy', np.array(original_question_embeddings))

## Store Quesrion-Query Pairs in VectorDB

In [ ]:
# Initialize ChromaDB client
client = chromadb.Client()

# Create a new collection in ChromaDB
collection = client.get_or_create_collection(name='final')


# Prepare data and store in ChromaDB
ids = []
embeddings = []
metadata = []

for idx, item in enumerate(spider_train_data):
    original_question = item['question']
    sql_query = item['query']
    db_id = item['db_id']



    skeleton_embedding = original_question_embeddings[idx]

    ids.append(str(idx))
    embeddings.append(skeleton_embedding.tolist())
    metadata.append({
        'original_question': original_question,
        'sql_query': sql_query,
        'db_id':db_id
    })

# Adjust metadata to ensure all values are strings
adjusted_metadata = []
for item in metadata:
    adjusted_metadata.append({
        'original_question': item['original_question'],
        'sql_query': item['sql_query'],
        'db_id': item['db_id']
    })

collection.add(
    ids=ids,
    embeddings=embeddings,
    metadatas=adjusted_metadata
)

## Retrieve Similar Examples

In [ ]:
# Retrieve similar queries
def retrieve_similar_queries(question, k=4):
    skeleton_embedding = get_openai_embedding(question)

    results = collection.query(
        query_embeddings=[skeleton_embedding.tolist()],
        n_results=k
    )

    #return results
    retrieved_metadata = [result for result in results['metadatas'][0]]

    return retrieved_metadata

In [ ]:
retrieve_similar_queries("Which employees were hired after September 7th, 1987?")

[{'db_id': 'hr_1',
  'original_question': 'Which employees were hired after September 7th, 1987?',
  'sql_query': "SELECT * FROM employees WHERE hire_date  >  '1987-09-07'"},
 {'db_id': 'hr_1',
  'original_question': 'display those employees who joined after 7th September, 1987.',
  'sql_query': "SELECT * FROM employees WHERE hire_date  >  '1987-09-07'"},
 {'db_id': 'hr_1',
  'original_question': 'What are the job ids and dates of hire for employees hired after November 5th, 2007 and before July 5th, 2009?',
  'sql_query': "SELECT job_id ,  hire_date FROM employees WHERE hire_date BETWEEN '2007-11-05' AND '2009-07-05'"},
 {'db_id': 'hr_1',
  'original_question': 'What is all the information about employees hired before June 21, 2002?',
  'sql_query': "SELECT * FROM employees WHERE hire_date  <  '2002-06-21'"}]

# Get all similar examples to test data

In [ ]:
import json
import numpy as np
from openai import OpenAI
import chromadb

In [ ]:
# Load the Spider dataset
train_path = "/content/drive/MyDrive/spider/spider/train_spider.json"
test_path = "/content/drive/MyDrive/spider/spider/test_data/dev.json"
with open(train_path, 'r') as f:
    spider_train_data = json.load(f)

with open(test_path, 'r') as f:
    spider_test_data = json.load(f)

In [ ]:
len(spider_test_data)

2147

In [ ]:
client_open_ai = OpenAI(api_key='OPEN_AI_API_KEY')

# Function to get embeddings using OpenAI
def get_openai_embedding(text):
    response = client_open_ai.embeddings.create(
                model="text-embedding-ada-002",
                input=text,
            )
    return np.array(response.data[0].embedding)

In [ ]:
# Load saved embeddings
save_path = "/content/drive/MyDrive/spider/"
original_question_embeddings = np.load(save_path + 'original_question_embeddings.npy')
original_question_embeddings = list(original_question_embeddings)

In [ ]:
# Initialize ChromaDB client
client = chromadb.Client()

# Create a new collection in ChromaDB
collection = client.get_or_create_collection(name='text-to-sql')


# Prepare data and store in ChromaDB
ids = []
embeddings = []
metadata = []

for idx, item in enumerate(spider_train_data):
    original_question = item['question']
    sql_query = item['query']
    db_id = item['db_id']



    embedding = original_question_embeddings[idx]

    ids.append(str(idx))
    embeddings.append(embedding.tolist())
    metadata.append({
        'original_question': original_question,
        'sql_query': sql_query,
        'db_id':db_id
    })

# Adjust metadata to ensure all values are strings
adjusted_metadata = []
for item in metadata:
    adjusted_metadata.append({
        'original_question': item['original_question'],
        'sql_query': item['sql_query'],
        'db_id': item['db_id']
    })

collection.add(
    ids=ids,
    embeddings=embeddings,
    metadatas=adjusted_metadata
)

In [ ]:
# Retrieve similar queries
def retrieve_similar_queries(question, k=4):
    skeleton_embedding = get_openai_embedding(question)

    results = collection.query(
        query_embeddings=[skeleton_embedding.tolist()],
        n_results=k
    )

    #return results
    retrieved_metadata = [result for result in results['metadatas'][0]]

    return skeleton_embedding, retrieved_metadata

In [ ]:
# removing the new line characters
with open('/content/drive/MyDrive/spider/results/sampled_tables_gpt3.5_v2.txt') as f:
    lines = [line.rstrip() for line in f]

In [ ]:
# Find similar questions for test data
#test_questions = [item['question'] for item in spider_test_data]

# Set k to the number of similar questions you want to retrieve
k = 5
test_similar_queries = []
test_emb = []
idx = 0
for item in lines:
    emb, similar_queries = retrieve_similar_queries(item, k)
    entry_dict = {
        "id" : idx,
        "similar_q" : similar_queries
    }
    idx +=1
    test_similar_queries.append(entry_dict)
    test_emb.append(emb)

In [ ]:
test_similar_queries[10]

{'id': 10,
 'similar_q': [{'db_id': 'soccer_1',
   'original_question': 'What is the average weight of all players?',
   'sql_query': 'SELECT avg(weight) FROM Player'},
  {'db_id': 'entrepreneur',
   'original_question': 'What are the names of people in ascending order of weight?',
   'sql_query': 'SELECT Name FROM People ORDER BY Weight ASC'},
  {'db_id': 'entrepreneur',
   'original_question': 'Return the names of people, ordered by weight ascending.',
   'sql_query': 'SELECT Name FROM People ORDER BY Weight ASC'},
  {'db_id': 'soccer_1',
   'original_question': 'What is the maximum and minimum height of all players?',
   'sql_query': 'SELECT max(weight) ,  min(weight) FROM Player'},
  {'db_id': 'candidate_poll',
   'original_question': 'how many people are there whose weight is higher than 85 for each gender?',
   'sql_query': 'SELECT count(*) ,  sex FROM people WHERE weight  >  85 GROUP BY sex'}]}

In [ ]:
# Save embeddings for later
save_path = "/content/drive/MyDrive/spider/"
np.save(save_path + 'test_question_embeddings.npy', np.array(test_emb))

In [ ]:
# Saving similar questions for test data
similarity_path = "/content/drive/MyDrive/spider/test_similar_questions/"
with open(similarity_path + 'revised_all_similar_queries.json', 'w') as fp:
    json.dump(test_similar_queries, fp)

In [ ]:
len(test_similar_queries)

2147

In [ ]:
# Saving similar questions for test data
similarity_path = "/content/drive/MyDrive/spider/test_similar_questions/"
with open(similarity_path + 'all_similar_questions.json', 'w') as fp:
    json.dump(test_similar_queries_dict, fp)

# Obtain similar samples for sampled test data

In [ ]:
import json

# Load the Spider dataset
test_path = "/content/drive/MyDrive/spider/sampled_dev.json"

with open(test_path, 'r') as f:
    spider_test_data = json.load(f)

In [ ]:
spider_test_data[0]

{'db_id': 'boat_1',
 'query': 'SELECT DISTINCT T1.name FROM Sailors AS T1 JOIN Reserves AS T2 ON T1.sid  =  T2.sid WHERE  T2.bid  =  103',
 'query_toks': ['SELECT',
  'DISTINCT',
  'T1.name',
  'FROM',
  'Sailors',
  'AS',
  'T1',
  'JOIN',
  'Reserves',
  'AS',
  'T2',
  'ON',
  'T1.sid',
  '=',
  'T2.sid',
  'WHERE',
  'T2.bid',
  '=',
  '103'],
 'query_toks_no_value': ['select',
  'distinct',
  't1',
  '.',
  'name',
  'from',
  'sailors',
  'as',
  't1',
  'join',
  'reserves',
  'as',
  't2',
  'on',
  't1',
  '.',
  'sid',
  '=',
  't2',
  '.',
  'sid',
  'where',
  't2',
  '.',
  'bid',
  '=',
  'value'],
 'question': 'Find the name of the sailors who reserved boat with id 103.',
 'question_toks': ['Find',
  'the',
  'name',
  'of',
  'the',
  'sailors',
  'who',
  'reserved',
  'boat',
  'with',
  'id',
  '103',
  '.'],
 'sql': {'from': {'table_units': [['table_unit', 0], ['table_unit', 2]],
   'conds': [[False, 2, [0, [0, 1, False], None], [0, 8, False], None]]},
  'select': [

In [ ]:
similar_test_path = "/content/drive/MyDrive/spider/test_similar_questions/revised_all_similar_questions.json"
with open(similar_test_path, 'r') as f:
    similar_data = json.load(f)

In [ ]:
len(similar_data)

2147

In [ ]:
sampled_similar_data = []
for entry in spider_test_data:
    for data in similar_data:
        q = data['question']
        db = data['db_id']
        if (q == entry['question']) and (db == entry['db_id']):
            sampled_similar_data.append(data)
            break

In [ ]:
sampled_similar_data

[{'question': 'Find the name of the sailors who reserved boat with id 103.',
  'db_id': 'boat_1',
  'similar_q': [{'db_id': 'ship_1',
    'original_question': 'Find the name of the ship that is steered by the youngest captain.',
    'sql_query': 'SELECT t1.name FROM ship AS t1 JOIN captain AS t2 ON t1.ship_id  =  t2.ship_id ORDER BY t2.age LIMIT 1'},
   {'db_id': 'ship_1',
    'original_question': 'Find the name of the ships that have more than one captain.',
    'sql_query': 'SELECT t1.name FROM ship AS t1 JOIN captain AS t2 ON t1.ship_id  =  t2.ship_id GROUP BY t2.ship_id HAVING count(*)  >  1'},
   {'db_id': 'ship_1',
    'original_question': 'Find the name of the ships that are steered by both a captain with Midshipman rank and a captain with Lieutenant rank.',
    'sql_query': "SELECT t1.name FROM ship AS t1 JOIN captain AS t2 ON t1.ship_id  =  t2.ship_id WHERE t2.rank  =  'Midshipman' INTERSECT SELECT t1.name FROM ship AS t1 JOIN captain AS t2 ON t1.ship_id  =  t2.ship_id WHERE t

In [ ]:
for data in similar_data:
    q = data['question']
    db = data['db_id']
    for entry in spider_test_data:
        if (q == entry['question']) and (db == entry['db_id']):
            sampled_similar_data.append(data)
            break

In [ ]:
len(sampled_similar_data)

300

In [ ]:
# Saving similar questions for test data
similarity_path = "/content/drive/MyDrive/spider/test_similar_questions/"

with open(similarity_path + 'sampled_similar_questions_v2.json', 'w') as fp:
    json.dump(sampled_similar_data, fp)